In [1]:
from __future__ import print_function
import os
import pickle
import numpy as np
import pandas as pd
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5004)

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')

In [3]:
w2indx = pickle.load(open('./0-output/LSTMDQN_Dic_w2idx.pkl', 'rb'))
indx2w = pickle.load(open('./0-output/LSTMDQN_Dic_idx2w.pkl', 'rb'))

In [4]:
len(w2indx), len(indx2w)

(129550, 129550)

In [5]:
adf = pd.read_csv('./0-output/2012_aurora_shooting_numtext.csv')

In [6]:
rans = np.random.random((adf.shape[0],1)) >0.5
X_train, X_test = adf.values[rans], adf.values[rans==False]

In [7]:
X_train = np.asarray([ [int(j) for j in i.split(" ")] for i in X_train])
X_test = np.asarray([ [int(j) for j in i.split(" ")] for i in X_test])

In [8]:
X_train.shape, X_test.shape

((104609,), (104097,))

In [9]:
X_train, X_test = X_train[0:10], X_test[0:10]

In [10]:
emb_dim = 10
batch_size = 32

# Taking the last element of X_train[i] as our outcome
Y_train = np.asarray([i[-1] for i in X_train]) 
Y_test = np.asarray([ i[-1] for i in X_test])

# Removing the last element of X_train[i]
X_train = np.asarray([i[:-1] for i in X_train]) 
X_test = np.asarray([i[:-1] for i in X_test] )

In [11]:
[x for x in X_train[0]]

[0, 7, 12, 6, 13, 10, 5, 2, 1, 9, 3, 11, 4]

In [14]:
print ([indx2w[x] for x in X_train[0]])

[u'A', u'Sikh', u'temple', u'is', u'harnessing', u'the', u'sun', u"'s", u'rays', u'to', u'supply', u'its', u'electricity']


In [15]:
print(indx2w[Y_train[0]])

.


In [17]:
maxlen=40

In [18]:
# RNN for the regular embedings
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
# The pad sequences stuff seems suspicious
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

10 train sequences
10 test sequences
Pad sequences (samples x time)
X_train shape: (10, 40)
X_test shape: (10, 40)


In [20]:
print('Build model...')
model = Sequential()
model.add(Embedding(129541, emb_dim)) # , dropout=0.2))
model.add(LSTM(150))
model.add(Dense(len(indx2w)))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

optimizer = RMSprop(lr=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1,
          validation_data=(X_test, Y_test), verbose=0)

# Running the predictions
out = model.evaluate(X_test, Y_test, batch_size=batch_size)
ypred_tst = model.predict(X_test)

Build model...
Train...
10/10 [==============================] - 0s


In [21]:
print('Cross Entropy Loss of %0.3f' % out)

yprdtst = np.argmax(ypred_tst, axis=1)
x = confusion_matrix(yprdtst, Y_test).astype('float') 
# Normalizing it
x_n = x/ x.sum(axis=0)

print ('Classification Accuracy is %0.3f' % (np.diag(x).sum()/ x.sum()))

Cross Entropy Loss of 11.380
Classification Accuracy is 0.300
